In [54]:
import pandas as pd
meta_data = pd.read_json("data/meta-North_Dakota.json", lines=True)
out_data = pd.read_json("data/out-North_Dakota.json", lines=True) 

meta_data[:5]

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,St Peter & Paul Church,"St Peter & Paul Church, 500 Main St, Karlsruhe...",0x52d94fbefa0e6353:0xf709e2d8674fe3a,None,48.093248,-100.618664,[Catholic church],4.9,7,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x52d9384b75abac93:0x13526f8266cae6cf, 0x52d9...",https://www.google.com/maps/place//data=!4m2!3...
1,Northwest Martial Arts Academy,"Northwest Martial Arts Academy, 1430 Main Ave,...",0x52c8cbe775edec7d:0xb46e15ed33643070,None,46.875093,-96.802717,[Martial arts school],5.0,8,None,"[[Thursday, 7:30AM–8PM], [Friday, 7:30AM–8PM],...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 7:30AM,"[0x52c8ccbcb1785327:0x2d50311eabd7afc, 0x52cf3...",https://www.google.com/maps/place//data=!4m2!3...
2,Thad's Amazing Magic - Fargo Birthday Party Magic,Thad's Amazing Magic - Fargo Birthday Party Ma...,0x52c8cd270f50bbbb:0x4ee4629598a8090e,None,46.812415,-96.856729,"[Magician, Children's party service]",5.0,58,None,"[[Thursday, Open 24 hours], [Friday, Open 24 h...",{'Amenities': ['Good for kids']},Open 24 hours,"[0x52c8c9613725e9ef:0xc628b86d8593e7e6, 0x52c8...",https://www.google.com/maps/place//data=!4m2!3...
3,Threefold,"Threefold, 212 W Main Ave, Bismarck, ND 58501",0x52d7836b7314da5d:0xc3cc63667b8c13a0,None,46.805707,-100.792990,"[Film production company, Video editing servic...",5.0,5,None,"[[Wednesday, 9AM–6PM], [Thursday, 9AM–6PM], [F...",None,Closed ⋅ Opens 9AM Thu,"[0x52d7836c2b519b77:0x74c84187e38f42b, 0x52d78...",https://www.google.com/maps/place//data=!4m2!3...
4,Gray Brothers Dairy,"Gray Brothers Dairy, 408 N Main St, Stanley, N...",0x5320bcc09c8e6f15:0xc888ebee3ea483b6,None,48.324312,-102.390000,None,5.0,1,None,None,None,None,"[0x5320bcc63e8fe69d:0x4f22ad0dd39b1970, 0x5320...",https://www.google.com/maps/place//data=!4m2!3...


In [55]:
# Drop columns for out_data
out_data = out_data.drop(['name', 'time', 'resp'], axis=1, errors="ignore")
# out_data.to_csv("data/clean_out_data.csv")
out_data.head()

,user_id,rating,text,pics,gmap_id
0,1.024707e+20,5,Fast quick service. I go here almost all the t...,None,0x52d0147640d529bd:0xec9407dc9201780b
1,1.022127e+20,5,Mike and crew do a great job. Dropped off veh...,None,0x52d0147640d529bd:0xec9407dc9201780b
2,1.125204e+20,5,Amazing love this place!,None,0x52d0147640d529bd:0xec9407dc9201780b
3,1.133982e+20,5,Fair prices speedy work,None,0x52d0147640d529bd:0xec9407dc9201780b
4,1.160142e+20,5,Always in & out. Super professional and conve...,None,0x52d0147640d529bd:0xec9407dc9201780b


In [56]:
# Create image index table
rows = []
for _, row in out_data[pd.notnull(out_data["pics"])].iterrows():
    text = row["text"]
    for pic in row["pics"]:
        urls = pic.get("url", [])
        for url in urls:
            rows.append({"urls": url, "text": text})

image_final_output = pd.DataFrame(rows)
image_final_output = image_final_output.drop_duplicates(subset=['urls'])
image_final_output[:10]

,urls,text
0,https://lh5.googleusercontent.com/p/AF1QipMvDk...,Highly recommend! These guys did a great job a...
2,https://lh5.googleusercontent.com/p/AF1QipOPEx...,Love love will be back again pedicure was amazing
4,https://lh5.googleusercontent.com/p/AF1QipPdmJ...,This gas station has the beat breakfast pizza ...
5,https://lh5.googleusercontent.com/p/AF1QipMuEQ...,This gas station has the beat breakfast pizza ...
6,https://lh5.googleusercontent.com/p/AF1QipPHCL...,Always lots of fun.
7,https://lh5.googleusercontent.com/p/AF1QipMdf5...,Always lots of fun.
8,https://lh5.googleusercontent.com/p/AF1QipN83T...,Always lots of fun.
9,https://lh5.googleusercontent.com/p/AF1QipO9Pt...,Always lots of fun.
10,https://lh5.googleusercontent.com/p/AF1QipPLCh...,Always lots of fun.
11,https://lh5.googleusercontent.com/p/AF1QipPjbv...,I love going here. Great food. Never a wait. N...


In [57]:
# Prune meta data
meta_data = meta_data[["gmap_id", "category"]]
final_output = pd.merge(meta_data, out_data, on="gmap_id", how="inner")


In [58]:
final_output = final_output.drop(['gmap_id', 'pics'], axis=1, errors="ignore")
final_output[:5]

,category,user_id,rating,text
0,[Auto repair shop],1.024707e+20,5,Fast quick service. I go here almost all the t...
1,[Auto repair shop],1.022127e+20,5,Mike and crew do a great job. Dropped off veh...
2,[Auto repair shop],1.125204e+20,5,Amazing love this place!
3,[Auto repair shop],1.133982e+20,5,Fair prices speedy work
4,[Auto repair shop],1.160142e+20,5,Always in & out. Super professional and conve...


In [59]:
image_final_output[:5]

,urls,text
0,https://lh5.googleusercontent.com/p/AF1QipMvDk...,Highly recommend! These guys did a great job a...
2,https://lh5.googleusercontent.com/p/AF1QipOPEx...,Love love will be back again pedicure was amazing
4,https://lh5.googleusercontent.com/p/AF1QipPdmJ...,This gas station has the beat breakfast pizza ...
5,https://lh5.googleusercontent.com/p/AF1QipMuEQ...,This gas station has the beat breakfast pizza ...
6,https://lh5.googleusercontent.com/p/AF1QipPHCL...,Always lots of fun.


In [60]:
final_output.to_csv("data/out-NDK-final.csv")
image_final_output.to_csv("data/image-index-NDK.final.csv")

In [61]:
# # Chunk large files
# import os

# # Load the large CSV
# df = pd.read_csv("data/image-index-NDK.final.csv")

# # Set chunk size (number of rows per chunk, adjust as needed)
# chunk_size = 150000  # You may need to adjust this to keep each file <100MB

# # Create output directory if it doesn't exist
# os.makedirs("data/image_chunks", exist_ok=True)

# # Split and save chunks
# for i in range(0, len(df), chunk_size):
#     chunk = df.iloc[i:i+chunk_size]
#     chunk.to_csv(f"data/image_chunks/image-index-NDK.final.part{i//chunk_size+1}.csv", index=False)